## Import libaries

In [ ]:
!pip uninstall dea_ml -y

In [ ]:
!pip install -e dea_ml
# pip install git+https://github.com/digitalearthafrica/deafrica-sandbox-notebooks.git@minty-fresh-sandbox#subdirectory=Tools
# !pip install git+https://github.com/digitalearthafrica/crop-mask/@feature/k8s-image-pipeline-tunning#subdirectory=production/dea_ml

In [1]:
!cm-pred run s3://deafrica-data-dev-af/crop_mask_eastern/0-1-0/gm_s2_semiannual_all.db --config=https://raw.githubusercontent.com/digitalearthafrica/crop-mask/feature/k8s-image-pipeline-tunning/production/dea_ml/dea_ml/config/plugin_product.yaml  --plugin-config=https://raw.githubusercontent.com/digitalearthafrica/crop-mask/main/production/dea_ml/dea_ml/config/ml_config.yaml  --resolution=20 --threads=10 --memory-limit=80Gi --location=file:///home/jovyan/git/crop-mask/production/{product}/{version} 4011:4012

/env/lib/python3.6/site-packages/geopandas/_compat.py:88: UserWarning: The Shapely GEOS version (3.7.2-CAPI-1.11.0 ) is incompatible with the GEOS version PyGEOS was compiled with (3.9.0-CAPI-1.16.2). Conversions between both will be slow.
  shapely_geos_version, geos_capi_version_string
[2021-05-12 04:26:29,506] {pred_run.py:150} INFO - Config overrides: {'filedb': 's3://deafrica-data-dev-af/crop_mask_eastern/0-1-0/gm_s2_semiannual_all.db', 'threads': 10, 'memory_limit': '80Gi', 'output_location': 'file:///home/jovyan/git/crop-mask/production/{product}/{version}'}
[2021-05-12 04:26:29,506] {pred_run.py:167} INFO - Using this config: TaskRunnerConfig(filedb='s3://deafrica-data-dev-af/crop_mask_eastern/0-1-0/gm_s2_semiannual_all.db', aws_unsigned=True, plugin='pred-gm-s2', plugin_config={'chirps_paths': ['s3://deafrica-data-dev-af/crop_mask_references/rainfall/CHPclim_jan_jun_cumulative_rainfall.tif', 's3://deafrica-data-dev-af/crop_mask_references/rainfall/CHPclim_jul_dec_cumulative_ra

In [ ]:
# import xarray as xr
# xr.open_rasterio('/g/data/crop_mask_eastern_data/crop_mask_eastern/v0.1.8/x210/y077/2019/crop_mask_eastern_x210_y077_2019_prob.tif').plot(size=10)

In [ ]:
# !ls /home/jovyan/wa/u23/data/crop_mask_eastern/v0.1.7/x+029/y+000/2019